In [ ]:
import numpy as np
from pandas import DataFrame, read_csv
# read_csv()
import os

BLOCK_SIZE = 20
FILTER_THRESHOLD = 4

In [ ]:
from collections import namedtuple, defaultdict


# Point = namedtuple('Point',['window', 'snp', 'bp'])

class Point:
    def __init__(self, snp, bp=0):
        self.snp, self.bp = snp, bp
    
    @property
    def window(self):
        return self.snp // BLOCK_SIZE

# class Specimen:
#     def __init__(self, ident, sequence)
#         ident, sequence
    
class Node:
    def __init__(self, ident, start, end, upstream=None, downstream=None, specimens=None):
        self.ident = ident
        self.start = start #Point()
        self.end = end #Point()
        # {nothing_node:501, Node: 38,  Node: 201, Node: 3}
        self.upstream = defaultdict(lambda: 0) if not upstream else upstream 
        # {Node: 38,  Node: 201, Node: 3}
        self.downstream = defaultdict(lambda: 0) if not downstream else downstream
        self.specimens = set() if specimens is None else specimens
        assert self.start is not None and self.end is not None, self.details()
        assert self.end.snp is not None or (self.end.snp is None and self.start.snp is None), self.details()
#        assert sum(self.upstream.values()) is len(self.specimens), self.details()
#        assert sum(self.downstream.values()) is len(self.specimens), self.details()
    
    def __len__(self):
        return len(self.specimens)
    
    def __repr__(self):
        return "N%s(%s, %s)" % (str(self.ident), str(self.start.snp), str(self.end.snp))
    
    def __hash__(self):
        return hash(self.ident) + hash(self.start.snp) + hash(self.end.snp)
    
    def details(self):
        return f"""Node{self.ident}: {self.start.snp} - {self.end.snp}
        upstream: { dict((key, value) for key,value in self.upstream.items()) }
        downstream: { dict((key, value) for key,value in self.downstream.items()) }
        specimens: {self.specimens}"""
        

a = Point(0)
b = Point(14)
str(Node(57, a, b))
nothing_node = Node(-1, Point(None), Point(None))
global_nodes = {0: nothing_node}


deepcopy(a)

In [ ]:
def read_data(file_path = "../test_data/KE_chromo10.txt"):
    """Individuals are rows, not columns"""
    loci = []
    with open(file_path) as ke:
        for line in ke.readlines():
            loci.append(tuple(int(x) for x in line.split()))
            
    
    individuals = np.array(loci).T.tolist()
    return loci, individuals
alleles, individuals = read_data()
assert len(alleles) == 32767
assert len(individuals[1]) == 32767
assert len(individuals) == 501

In [ ]:
def first(iterable):
    return next(iter(iterable))

In [ ]:
def signature(individual, start_locus):
    return tuple(individual[start_locus : start_locus + BLOCK_SIZE])

def get_unique_signatures(individuals, start_locus, block_size = 20):
    unique_blocks = {}
    for individual in individuals:
        sig = signature(individual, start_locus)
        if sig not in unique_blocks:
            unique_blocks[sig] = Node(len(unique_blocks), Point(start_locus // block_size, start_locus), 
                                      Point(start_locus // block_size, start_locus + BLOCK_SIZE)) #TODO: -1?
    
    return unique_blocks

def test_get_unique_signatures(individuals):
    unique_blocks = get_unique_signatures(individuals, 0 )
    assert len(unique_blocks) == 4
    assert unique_blocks.__repr__() == '{(0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0): N0(0, 0), '\
    '(0, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2): N1(0, 0), '\
    '(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): N2(0, 0), '\
    '(2, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2): N3(0, 0)}'
test_get_unique_signatures(individuals)

In [ ]:
def get_all_signatures(alleles, individuals):
    unique_signatures = []
    for locus_start in range(0, len(alleles) - BLOCK_SIZE, BLOCK_SIZE):  # discards remainder 
        sig = get_unique_signatures(individuals, locus_start, BLOCK_SIZE)
        unique_signatures.append(sig)
    return unique_signatures

In [ ]:
def build_individuals(individuals, unique_signatures):
    simplified_individuals = []
    for i_specimen, specimen in enumerate(individuals):
        my_simplification = []
        for w, window in enumerate(unique_signatures):  # the length of the genome
            sig = signature(specimen, w * BLOCK_SIZE)
    #         print(sig, unique_signatures[w][sig])
    #         print(i_specimen, window)
            my_simplification.append(unique_signatures[w][sig])
        simplified_individuals.append(my_simplification)
    return simplified_individuals

def test_build_individuals(alleles, individuals):
    unique_signatures = get_all_signatures(alleles, individuals)
    assert repr(unique_signatures[21]) == '{(0, 0, 0, 0, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2): N0(21, 21), (0, 0, 2, 0, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2): N1(21, 21), (0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): N2(21, 21), (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): N3(21, 21), (0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): N4(21, 21), (0, 0, 0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 2): N5(21, 21), (0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): N6(21, 21), (0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 2, 2): N7(21, 21)}'
    simplified_individuals = build_individuals(individuals, unique_signatures)
    assert repr(simplified_individuals[500][:100]) == '[N2(0, 0), N2(1, 1), N2(2, 2), N2(3, 3), N2(4, 4), N2(5, 5), N3(6, 6), N3(7, 7), N3(8, 8), N2(9, 9), N0(10, 10), N1(11, 11), N2(12, 12), N2(13, 13), N2(14, 14), N2(15, 15), N3(16, 16), N3(17, 17), N4(18, 18), N3(19, 19), N5(20, 20), N3(21, 21), N3(22, 22), N10(23, 23), N4(24, 24), N3(25, 25), N4(26, 26), N3(27, 27), N1(28, 28), N1(29, 29), N4(30, 30), N3(31, 31), N21(32, 32), N1(33, 33), N1(34, 34), N1(35, 35), N1(36, 36), N1(37, 37), N1(38, 38), N1(39, 39), N1(40, 40), N1(41, 41), N1(42, 42), N1(43, 43), N1(44, 44), N1(45, 45), N1(46, 46), N1(47, 47), N1(48, 48), N1(49, 49), N1(50, 50), N1(51, 51), N1(52, 52), N1(53, 53), N1(54, 54), N1(55, 55), N1(56, 56), N1(57, 57), N1(58, 58), N1(59, 59), N1(60, 60), N1(61, 61), N1(62, 62), N1(63, 63), N1(64, 64), N1(65, 65), N1(66, 66), N1(67, 67), N1(68, 68), N1(69, 69), N1(70, 70), N1(71, 71), N1(72, 72), N1(73, 73), N1(74, 74), N1(75, 75), N1(76, 76), N1(77, 77), N0(78, 78), N0(79, 79), N1(80, 80), N1(81, 81), N1(82, 82), N1(83, 83), N1(84, 84), N1(85, 85), N1(86, 86), N1(87, 87), N1(88, 88), N1(89, 89), N1(90, 90), N1(91, 91), N1(92, 92), N1(93, 93), N1(94, 94), N1(95, 95), N1(96, 96), N1(97, 97), N0(98, 98), N0(99, 99)]'
    assert len(simplified_individuals) == 501 and len(simplified_individuals[60]) == 1638
test_build_individuals(alleles, individuals)

# Nodes: Populate upstream and downstream

In [ ]:
# build nodes:  first 4 are the 4 starting signatures in window 0.  
# For each node list which individuals are present at that node
# List transition rates from one node to all other upstream and downstream
def populate_transitions(simplified_individuals):
    for i, indiv in enumerate(simplified_individuals):
        # look what variants are present
        for x, node in enumerate(indiv):
            node.specimens.add(i)
            if x + 1 < len(indiv):
                node.downstream[indiv[x+1]] += 1
            else:
                node.downstream[nothing_node] += 1
            if x-1 >= 0:
                node.upstream[indiv[x-1]] += 1
            else: 
                node.upstream[nothing_node] += 1
            

In [ ]:
unique_signatures = get_all_signatures(alleles, individuals)
simplified_individuals = build_individuals(individuals, unique_signatures)
populate_transitions(simplified_individuals)

#### TODO: turn these into tests

In [ ]:
simplified_individuals[50][0].downstream

In [ ]:
simplified_individuals[49][0].downstream

In [ ]:
simplified_individuals[500][0].downstream

In [ ]:
simplified_individuals[91][0].downstream

In [ ]:
[x.downstream.values() for x in unique_signatures[1000].values()]

In [ ]:
[x.upstream.values() for x in unique_signatures[1000].values()]

---------------

# Simple Merge

In [ ]:
# TODO: add signature directly to node

In [ ]:
#from blist import blist
blist = list
from copy import copy, deepcopy

In [ ]:
def test_no_duplicate_nodes(global_nodes):
    unique_nodes = set()
    for node in global_nodes:
        if node in unique_nodes:
            print(node)
        else:
            unique_nodes.add(node)


In [ ]:
# zoom_stack = [[]]
def simple_merge(global_nodes):
    new_layer = []  # TODO: copy old nodes to new layer conditionally
    n = 0
    while n < len(global_nodes):  # size of global_nodes changes, necessitating this weird loop
        node = global_nodes[n]
    #     print(node, type(node))
        if len(node.downstream) == 1: 
            next_node = first(node.downstream.keys())
            if len(node.specimens) == len(next_node.specimens):
                #Torsten deletes nodeA and modifies next_node
                next_node.upstream = node.upstream
                next_node.start = node.start
                #prepare to delete node by removing references
                for parent in node.upstream.keys():
                    if parent != nothing_node:
                        count = parent.downstream[node]
                        del parent.downstream[node]  # updating pointer 
                        parent.downstream[next_node] = count 
                global_nodes.remove(node)  #delete node
                # zoom_stack[0].append(merged)
                n -= 1
        n += 1
    return global_nodes        

In [ ]:
def test_simple_merge(unique_signatures):
    global_nodes = blist([node for window in unique_signatures for node in window.values()])  # think about referencing and deletion
    assert len(global_nodes) == 7180
    summary1 = simple_merge(global_nodes)
    assert len(summary1) == 3690
    return summary1
summary1 = test_simple_merge(unique_signatures)

#### Neglect Nodes

In [ ]:
def delete_node(node, cutoff):
    """Changes references to this node to add to references to nothing_node"""
    if cutoff < 1:
        return  # if cutoff is 0, then don't touch upstream and downstream
    for parent, count in node.upstream.items():
        parent.downstream[nothing_node] += parent.downstream[node]
        del parent.downstream[node]
    for descendant, count in node.downstream.items():
        descendant.upstream[nothing_node] += descendant.upstream[node]
        del descendant.upstream[node]
        

def neglect_nodes(all_nodes, deletion_cutoff=FILTER_THRESHOLD):
    nodes_to_delete = set()
#     filtered_nodes = copy(all_nodes)
#     filtered_nodes.remove(1)
#     assert len(all_nodes) != len(filtered_nodes)
    for node in all_nodes:
        if len(node.specimens) <= deletion_cutoff:
            delete_node(node, deletion_cutoff)  # TODO: check if this will orphan 
            nodes_to_delete.add(node)
    filtered_nodes = blist([x for x in all_nodes if x not in nodes_to_delete])
    # TODO: remove orphaned haplotypes in a node that transition to and from zero within a 10 window length
    return filtered_nodes 


def test_neglect_nodes(all_nodes):
    summary2 = neglect_nodes(all_nodes)
    assert len(summary2) == 2854
    unchanged = neglect_nodes(summary2, 0)
    assert len([n for n in unchanged if len(n.specimens) == 0]) == 0
    return summary2
summary2 = test_neglect_nodes(summary1)

In [ ]:
summary2[5].details()

#### Split Groups

In [ ]:
def split_one_group(prev_node, anchor, next_node):
    """ Called when up.specimens == down.specimens"""
    # Comment: That is actually the case we want to split up to obtain longer blocks later
    # Extension of full windows will take care of potential loss of information later
    my_specimens = copy(anchor.specimens)
    if prev_node is not nothing_node:  # normal case
        my_specimens = my_specimens.intersection(prev_node.specimens)
    if next_node is not nothing_node:  # normal case
        my_specimens = my_specimens.intersection(next_node.specimens)
    if prev_node is nothing_node and next_node is nothing_node:  # exceptional: both are nothing node
        my_specimens = copy(anchor.specimens)
        for n in anchor.downstream.keys():
            if n is not nothing_node:  # don't remove empty set
                my_specimens -= n.specimens
        for n in anchor.upstream.keys():
            if n is not nothing_node:  # don't remove empty set
                my_specimens -= n.specimens
    
    my_start, my_end = prev_node.start, next_node.end
    my_upstream, my_downstream = prev_node.upstream, next_node.downstream
    if nothing_node is prev_node:  # Rare case
        my_start = anchor.start
        my_upstream = anchor.upstream
    if nothing_node is next_node:  # Rare case
        my_end = anchor.end
        my_downstream = anchor.downstream
        
    # TODO: what about case where more content is joining downstream?
    new = Node(777, my_start, my_end, my_upstream, my_downstream, my_specimens)  
    
    
    ## n.upstream/downstream contains the same key multiple times?!
    ## My quick fix was to delete all upstream/downstream and just recalculate everything...
    
    
    # Update upstream/downstream + specimens in prev_node, anchor, next_node
    if prev_node != nothing_node:
        prev_node.specimens -= new.specimens
        prev_node = update_transition(prev_node)
    
    if next_node != nothing_node:
        next_node.specimens -= new.specimens
        next_node = update_transition(next_node)
    
    anchor.specimens -= new.specimens
    anchor = update_transition(anchor)
    
    # Update upstream/downstream for new node
    running = tuple(new.upstream.keys())
    #new.upstream = defaultdict(lambda: 0) 
    for n in running:
        if n != nothing_node:
            new.upstream[n] = len(new.specimens.intersection(n.specimens))
            n.downstream[new] = new.upstream[n]
            update_transition(n)
        update_transition(new)
    
    
    running = tuple(new.downstream.keys())
    #new.downstream = defaultdict(lambda: 0)
    for n in running:
        if n != nothing_node:
            new.downstream[n] = len(new.specimens.intersection(n.specimens))
            n.upstream[new] = new.downstream[n]
            update_transition(n)
        update_transition(new)
    
    
    assert all([count > -1 for count in new.upstream.values()]), new.details()
    assert all([count > -1 for count in new.downstream.values()]), new.details()

    
    ## anchor.specimens.difference_update(prev_node.specimens) REASON?
    return new

In [ ]:
# Needed?
def update_transition(node):
    if node is not nothing_node:
        running = node.upstream.keys()
        node.upstream = defaultdict(lambda: 0)
        for n in running:
            if n is not nothing_node:
                node.upstream[n] = len(node.specimens.intersection(n.specimens))
             
        running = node.downstream.keys()
        node.downstream = defaultdict(lambda: 0)
        for n in running:
            if n is not nothing_node:
                node.downstream[n] = len(node.specimens.intersection(n.specimens))
                
        accounted_upstream = sum(node.upstream.values()) - node.upstream[nothing_node]
        node.upstream[nothing_node] = len(node.specimens) - accounted_upstream
        accounted_downstream = sum(node.downstream.values()) - node.downstream[nothing_node]
        node.downstream[nothing_node] = len(node.specimens) - accounted_downstream 

    assert sum(node.upstream.values()) == len(node.specimens), node.details()
    
    return node

        
    

test_graph = summary2  # deepcopy(
example = test_graph[7]
original = deepcopy(example)
print(example.details())
def test_split_one_group(prev_node, anchor, next_node):
    x = split_one_group(prev_node, anchor, next_node)
    assert x
    answer = set(int(x)-1 for x in '14  16  19  20  28  56  59  69  88 133 140 155 159 160 175 193 199 201 224 249 252 258 260 267 268 283 292 295 318 322 325 332 341 344 346 351 354 357 362 364 367 373 374 375 381 386 392 393 394 402 403 417 421 424 426 431 434 435 438 442 445 447 452 455 457 462 463 464 467 471 473 475 476 477 478 480 483 484 494 497 501'.split())
    assert x.specimens == answer, 'Specimens set does not agree with HaploBlocker' + str(x.specimens.difference(answer))
    return x

x = test_split_one_group(first(example.upstream),  example, first(example.downstream) )

example.details()

original[7].details()

In [ ]:
def split_groups(all_nodes):
    """This is called crossmerge in the R code"""
    number_of_windows = len(first(simplified_individuals))
    length = len(all_nodes)# size of global_nodes changes, necessitating this weird loop
    for n in range(length):  
        node = all_nodes[n]
        #check if all transitition upstream match with one of my downstream nodes
        #if set(node.upstream.values()) == set(node.downstream.values()): WHY?
        if node.start.snp != 0 and node.end.window != number_of_windows: #chr begin or end
            if len(node.specimens) > 0:
                # Matchup upstream and downstream with specimen identities
                for up in tuple(node.upstream.keys()):
                    for down in tuple(node.downstream.keys()):

                        set1 = up.specimens
                        set2 = down.specimens
                        if up == nothing_node:
                            set1 = node.specimens
                            for index in tuple(node.upstream.keys()):
                                if index != nothing_node:
                                    set1.difference_update(index.specimens)
                        if down == nothing_node:
                            set2 = node.specimens
                            for index in tuple(node.downstream.keys()):
                                if index != nothing_node:
                                    set2.difference_update(index.specimens) 
                        if set1 == set2 and len(set1) > 0:
                            new_node = split_one_group(up, node, down) # This changes all_nodes?!
                            all_nodes.append(new_node)
        
    filtered = neglect_nodes(all_nodes, 0)
    

    length2 = len(filtered)
    for n in range(length2):
        update_transition(filtered[n])
    
    return filtered
        

In [ ]:
def split_one_group(prev_node, anchor, next_node):
    """ Called when up.specimens == down.specimens"""
    # Comment: That is actually the case we want to split up to obtain longer blocks later
    # Extension of full windows will take care of potential loss of information later
    my_specimens = anchor.specimens
    if prev_node is not nothing_node:  # normal case
        my_specimens = my_specimens.intersection(prev_node.specimens)
    if next_node is not nothing_node:  # normal case
        my_specimens = my_specimens.intersection(next_node.specimens)
    if prev_node is nothing_node and next_node is nothing_node:  # exceptional: both are nothing node
        my_specimens = anchor.specimens
        for n in anchor.downstream.keys():
            if n is not nothing_node:  # don't remove empty set
                my_specimens -= n.specimens
        for n in anchor.upstream.keys():
            if n is not nothing_node:  # don't remove empty set
                my_specimens -= n.specimens
    
    my_start, my_end = prev_node.start, next_node.end
    my_upstream, my_downstream = prev_node.upstream, next_node.downstream
    if nothing_node is prev_node:  # Rare case
        my_start = anchor.start
        my_upstream = anchor.upstream
    if nothing_node is next_node:  # Rare case
        my_end = anchor.end
        my_downstream = anchor.downstream
        
    # TODO: what about case where more content is joining downstream?
    new = Node(777, my_start, my_end, my_upstream, my_downstream, my_specimens)  
    new.specimens = my_specimens
    
    


    ## n.upstream/downstream contains the same key multiple times?!
    ## My quick fix was to delete all upstream/downstream and just recalculate everything...
    
    
    # Update upstream/downstream + specimens in prev_node, anchor, next_node
    if prev_node != nothing_node:
        prev_node.specimens -= new.specimens
        prev_node = update_transition(prev_node)
    
    if next_node != nothing_node:
        next_node.specimens -= new.specimens
        next_node = update_transition(next_node)
    
    anchor.specimens -= new.specimens
    anchor = update_transition(anchor)
    
    # Update upstream/downstream for new node
    running = new.upstream.keys()
    new.upstream = defaultdict(lambda: 0) 
    for n in running:
        if n != nothing_node:
            new.upstream[n] = len(new.specimens.intersection(n.specimens))
            n.downstream[new] = new.upstream[n]
            update_transition(n)
        update_transition(new)
    
    
    running = new.downstream.keys()
    new.downstream = defaultdict(lambda: 0)
    for n in running:
        if n != nothing_node:
            new.downstream[n] = len(new.specimens.intersection(n.specimens))
            n.upstream[new] = new.downstream[n]
            update_transition(n)
        update_transition(new)
    
    
    assert all([count > -1 for count in new.upstream.values()]), new.details()
    assert all([count > -1 for count in new.downstream.values()]), new.details()

    
    ## anchor.specimens.difference_update(prev_node.specimens) REASON?
    return new

In [ ]:
def update_transition(node):
    if node is not nothing_node:
        running = node.upstream.keys()
        node.upstream = defaultdict(lambda: 0)
        for n in running:
            if n is not nothing_node:
                node.upstream[n] = len(node.specimens.intersection(n.specimens))
             
        running = node.downstream.keys()
        node.downstream = defaultdict(lambda: 0)
        for n in running:
            if n is not nothing_node:
                node.downstream[n] = len(node.specimens.intersection(n.specimens))
                
        accounted_upstream = sum(node.upstream.values()) - node.upstream[nothing_node]
        node.upstream[nothing_node] = len(node.specimens) - accounted_upstream
        accounted_downstream = sum(node.downstream.values()) - node.downstream[nothing_node]
        node.downstream[nothing_node] = len(node.specimens) - accounted_downstream 
    return node

In [ ]:
def test_split_groups(all_nodes):
    summary3 = split_groups(all_nodes)
    assert summary3
    return summary3
summary3 = test_split_groups(summary2)

In [ ]:
len(summary2), len(summary3)

In [ ]:
print(summary3[1400].details())

# All Executions Necessary for testing

In [ ]:
# All Executions Necessary for testing without side effects
# alleles, individuals = read_data()

test_signatures = get_all_signatures(alleles, individuals)
unique_signatures = test_signatures
test_individuals = build_individuals(individuals, test_signatures)
populate_transitions(test_individuals) # no return val

test1 = test_simple_merge(test_signatures)
test2 = neglect_nodes(test1)
test3 = split_groups(test2)

In [ ]:
len(test1), len(test2), len(test3)

In [ ]:
test_individuals[1]

In [ ]:
len(summary3) ## Order of nodes does matter here! HaploBlocker output: 1887

### Everything below does not work currently. Some operations on empty sets lead to crashes. 
#### We should clean up upstream/downstream to not display 0 transition cases. del did not work directly

### Simple-merge / Cross-merge runs

In [ ]:
test_signatures = get_all_signatures(alleles, individuals)
unique_signatures = test_signatures
test_individuals = build_individuals(individuals, test_signatures)
populate_transitions(test_individuals) # no return val

window_cluster = test_simple_merge(test_signatures)

In [ ]:
window_cluster = split_groups(window_cluster)

In [ ]:
print(len(window_cluster)) ### HB outout after first iteration is 2504 nodes
# but we are here merging more so slightly less makes sense

In [ ]:
window_cluster = simple_merge(window_cluster)
print(len(window_cluster))


In [ ]:
### SOMETHINGS GOES TERRIBLY WRONG HERE
window_cluster = split_groups(window_cluster)
print(len(window_cluster))


In [ ]:
for i in range(10):
    window_cluster = split_groups(window_cluster)
    window_cluster = simple_merge(window_cluster)
    print(len(window_cluster))
    # FINAL value in R-HB here: 2333

### Neglect nodes runs

In [ ]:
for i in range(10):
    window_cluster = neglect_nodes(window_cluster)
    window_cluster = simple_merge(window_cluster)
    window_cluster = split_groups(window_cluster)
    window_cluster = simple_merge(window_cluster)
    print(len(window_cluster))
    # FINAL value in R-HB here: 1585

In [ ]:
window_cluster

In [ ]:
window_cluster = blist([node for window in unique_signatures for node in window.values()])  # think about referencing and deletion
window_cluster = simple_merge(window_cluster)
#window_cluster = split_groups(window_cluster)
#len(window_cluster)
#window_cluster[0].details()

In [ ]:
window_cluster[4].details()